# Julia 機器學習：LIBSVM 支撐向量機

## 作業 029：

請使用 SVM 來建立一個預測模型，來對手寫數字辨識資料集，建立一個分類模型，預測數字為何。

本作業需要使用到的套件有 MLDatasets，請在執行以下範例前先安裝。

```
] add MLDatasets
```

In [50]:
using LIBSVM, MLDatasets, Statistics

## 讀取資料

呼叫 MNIST 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。

In [51]:
train_X, train_y = MNIST.traindata(1:10000)
test_X,  test_y  = MNIST.testdata(1:1000)

(FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

...

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; … ; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8]

FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N

## 資料轉換

In [52]:
train_X = Matrix(reshape(train_X,(10000,28*28))) # convert to the shape that fits model
test_X = Matrix(reshape(test_X,(1000,28*28))) # convert to the shape that fits model

1000×784 Array{N0f8,2} with eltype FixedPointNumbers.Normed{UInt8,8}:
 0.0    0.0  0.0    0.0    0.0    0.0    …  0.0    0.0    0.0    0.0    0.0
 0.0    0.0  0.671  0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.502
 0.0    0.0  0.996  0.0    0.0    0.408     0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.318  0.0    0.0    0.996     0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.0    0.0    0.62      0.173  0.0    0.0    0.992  0.988
 0.0    0.0  0.0    0.0    0.0    0.0    …  0.898  0.0    0.0    0.984  0.992
 0.0    0.0  0.0    0.282  0.0    0.0       0.992  0.0    0.0    0.984  0.988
 0.0    0.0  0.0    0.682  0.0    0.0       0.494  0.0    0.0    0.122  0.988
 0.0    0.0  0.0    0.984  0.0    0.0       0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.678  0.0    0.0       0.0    0.0    0.0    0.0    0.992
 0.0    0.0  0.0    0.278  0.0    0.0    …  0.0    0.0    0.0    0.0    0.988
 0.0    0.0  0.0    0.282  0.0    0.0       0.0    0.0    0.0    0.0    0.

In [53]:
train_y = Vector(string.(train_y)) # convert to string type
test_y = Vector(string.(test_y)) # convert to string type

1000-element Array{String,1}:
 "7"
 "2"
 "1"
 "0"
 "4"
 "1"
 "4"
 "9"
 "5"
 "9"
 "0"
 "6"
 "9"
 ⋮
 "1"
 "6"
 "2"
 "3"
 "9"
 "0"
 "1"
 "2"
 "2"
 "0"
 "8"
 "9"

In [64]:
model = LIBSVM.fit!(SVC(),train_X,train_y)

SVC(LIBSVM.Kernel.RadialBasis, 0.0012755102040816326, nothing, 1.0, 3, 0.0, 0.001, true, false, false, LIBSVM.SVM{String}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 784, 10, ["5", "0", "4", "1", "9", "2", "3", "6", "7", "8"], Int32[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], Float64[], Int32[], LIBSVM.SupportVectors{String,FixedPointNumbers.Normed{UInt8,8}}(9999, Int32[863, 1001, 980, 1126, 978, 991, 1032, 1014, 1070, 944], ["5", "5", "5", "5", "5", "5", "5", "5", "5", "5"  …  "8", "8", "8", "8", "8", "8", "8", "8", "8", "8"], FixedPointNumbers.Normed{UInt8,8}[0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.498N0f8 … 0.671N0f8 0.0N0f8; … ; 0.816N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8; 0.0N0f8 0.0N0f8 … 0.0N0f8 0.0N0f8], Int32[1, 12, 36, 48, 66, 101, 133, 139, 146, 174  …  9936, 9937, 9938, 9945, 9949, 9953, 9957, 9971, 9983, 9997], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0), LIBSVM.SVMNode(1, 0.0),

## 預測

In [67]:
ŷ = String[]
for i = 1:size(test_X, 1)
    y_ = LIBSVM.predict(model, Matrix(test_X[i, :]'))
    append!(ŷ, y_)
end

## 評估

In [75]:
accuracy() = mean((ŷ .== test_X))*100

accuracy (generic function with 1 method)

In [76]:
accuracy()

0.0